<a href="https://colab.research.google.com/github/stepsbtw/Algoritmos/blob/main/algorithms_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Leitura e Filtragem inicial dos dados
Carregue os dados em sua linguagem de programação preferida (por exemplo, Python com pandas).

In [ ]:
import pandas as pd
import requests
from io import BytesIO

r = requests.get("https://data.brasil.io/dataset/covid19/caso.csv.gz")
caso = pd.read_csv(BytesIO(r.content), compression="gzip")

r = requests.get("https://data.brasil.io/dataset/covid19/caso_full.csv.gz")
caso_full = pd.read_csv(BytesIO(r.content), compression="gzip")

In [ ]:
caso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838003 entries, 0 to 2838002
Data columns (total 13 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   date                            object 
 1   state                           object 
 2   city                            object 
 3   place_type                      object 
 4   confirmed                       int64  
 5   deaths                          int64  
 6   order_for_place                 int64  
 7   is_last                         bool   
 8   estimated_population_2019       float64
 9   estimated_population            float64
 10  city_ibge_code                  float64
 11  confirmed_per_100k_inhabitants  float64
 12  death_rate                      float64
dtypes: bool(1), float64(5), int64(3), object(4)
memory usage: 262.5+ MB


In [ ]:
caso_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853648 entries, 0 to 3853647
Data columns (total 18 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   city                                           object 
 1   city_ibge_code                                 float64
 2   date                                           object 
 3   epidemiological_week                           int64  
 4   estimated_population                           float64
 5   estimated_population_2019                      float64
 6   is_last                                        bool   
 7   is_repeated                                    bool   
 8   last_available_confirmed                       int64  
 9   last_available_confirmed_per_100k_inhabitants  float64
 10  last_available_date                            object 
 11  last_available_death_rate                      float64
 12  last_available_deaths                     

Descrição das colunas:

- **city: nome do município (pode estar em branco quando o registro é referente ao estado, pode ser preenchido com Importados/Indefinidos também).**
- city_ibge_code: código IBGE do local.
- **date: data de coleta dos dados no formato YYYY-MM-DD.**
- epidemiological_week: número da semana epidemiológica no formato YYYYWW.
- estimated_population: população estimada para esse município/estado em 2020, segundo o IBGE. (acesse o script que faz o download e conversão dos dados de população).
- estimated_population_2019: população estimada para esse município/estado em 2019, segundo o IBGE. ATENÇÃO: essa coluna possui valores desatualizados, prefira usar a coluna estimated_population.
- is_last: campo pré-computado que diz se esse registro é o mais novo para esse local, pode ser True ou False (caso filtre por esse campo, use is_last=True ou is_last=False, não use o valor em minúsculas).
- is_repeated: campo pré-computado que diz se as informações nesse registro foram publicadas pela Secretaria Estadual de Saúde no dia date ou se o dado é repetido do último dia em que o dado está disponível (igual ou anterior a date). Isso ocorre pois nem todas as secretarias publicam boletins todos os dias. Veja também o campo last_available_date.
- **last_available_confirmed: número de casos confirmados do último dia disponível igual ou anterior à data date.**
- **last_available_confirmed_per_100k_inhabitants: número de casos confirmados por 100.000 habitantes (baseado em estimated_population) do último dia disponível igual ou anterior à data date.**
- last_available_date: data da qual o dado se refere.
- **last_available_death_rate: taxa de mortalidade (mortes / confirmados) do último dia disponível igual ou anterior à data date.**
- **last_available_deaths: número de mortes do último dia disponível igual ou anterior à data date.**
- order_for_place: número que identifica a ordem do registro para este local. O registro referente ao primeiro boletim em que esse local aparecer será contabilizado como 1 e os demais boletins incrementarão esse valor.
- place_type: tipo de local que esse registro descreve, pode ser city ou state.
- **state: sigla da unidade federativa, exemplo: SP.**
- new_confirmed: número de novos casos confirmados desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).
- new_deaths: número de novos óbitos desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).

Recomendadas para análise:
- city
- state
- last_avaible_confirmed
- last_available_deaths
- estimated_population
- last_available_confirmed_per_100k_inhabitants
- last_available_death_rate
- date

Para análises estatísticas mais simples:

In [ ]:
# Uma linha por município
df = caso[(caso["is_last"]==True) & (caso["place_type"]=="city")]

Para análise de desempenho de algoritmos com volumes maiores de dados (várias datas por município):

In [ ]:
# Séries temporais completas.
df_full = caso_full[(caso_full["place_type"]=="city") & (caso_full["is_repeated"]==False)]

Checar e tratar (remover linhas) colunas com valores nulos.

In [ ]:
df.count()

,0
date,5589
state,5589
city,5589
place_type,5589
confirmed,5589
deaths,5589
order_for_place,5589
is_last,5589
estimated_population_2019,5570
estimated_population,5570


In [ ]:
df_clean = df.dropna(subset=["confirmed_per_100k_inhabitants", "estimated_population"])
df_full.count()

,0
city,2818241
city_ibge_code,2807856
date,2818241
epidemiological_week,2818241
estimated_population,2807856
estimated_population_2019,2807856
is_last,2818241
is_repeated,2818241
last_available_confirmed,2818241
last_available_confirmed_per_100k_inhabitants,2793772


In [ ]:
df_full_clean = df_full.dropna(subset=["last_available_confirmed_per_100k_inhabitants", "estimated_population"])

## 2. Cálculo de Métricas
Para cada município:
- Taxa de casos por 100 mil habitantes:
  - casos/habitantes * 100 000
- Taxa de óbitos por 100 mil habitantes:
  - obitos/habitantes * 100 000
- Taxa de mortalidade:
  - obitos/casos

In [ ]:
# Atenção! Existem muitos municípios com o mesmo nome em estados diferentes:
counts = df_clean["city"].value_counts()
counts[counts>1]

,count
city,
São Domingos,5
Bom Jesus,5
Planalto,4
São Francisco,4
Vera Cruz,4
...,...
Petrolândia,2
Tabatinga,2
São Carlos,2


Portanto vamos usar o indentificador único do ibge.

In [ ]:
df_resposta = df_clean[["city", "state", "city_ibge_code"]].copy()

df_resposta["taxa_casos_100k"] = df_clean["confirmed"] / df_clean["estimated_population"] * 10**5
df_resposta["taxa_obitos_100k"] = df_clean["deaths"] / df_clean["estimated_population"] * 10**5
df_resposta["taxa_mortalidade"] = df_clean["deaths"] / df_clean["confirmed"]

df_resposta

,city,state,city_ibge_code,taxa_casos_100k,taxa_obitos_100k,taxa_mortalidade
734,Amapá,AP,1600105.0,15500.163274,141.504300,0.009129
1320,Calçoene,AP,1600204.0,16380.682823,123.828056,0.007559
1901,Cutias,AP,1600212.0,14325.520406,114.735289,0.008009
2478,Ferreira Gomes,AP,1600238.0,19656.081336,87.862433,0.004470
3058,Itaubal,AP,1600253.0,9791.703756,53.409293,0.005455
...,...,...,...,...,...,...
2835558,Águas de São Pedro,SP,3500600.0,13092.871343,340.812269,0.026030
2836080,Álvares Florence,SP,3501202.0,17164.792981,438.716753,0.025559
2836553,Álvares Machado,SP,3501301.0,10028.802304,316.025282,0.031512
2837057,Álvaro de Carvalho,SP,3501400.0,5536.594615,170.648464,0.030822


## 3. Ordenação:
Implemente os algoritmos de ordenação abaixo e aplique-os para classificar os municípios por:
- Maior taxa de casos por 100 mil hab.
- Maior taxa de mortalidade.
- Menor taxa de mortalidade.

Compare:
- Merge Sort
- Quick Sort
- Função nativa da linguagem (ex: sort() do Python)

In [ ]:
def mergesort(v, l=0, r=None, compare=(lambda x, y: x < y)):
  if r == None:
    r = len(v)-1
  # base case
  if l == r:
    return [v[l]]
  # divide T(n//2)
  m = (l+r)//2
  vL = mergesort(v, l, m, compare)
  vR = mergesort(v, m+1, r, compare)
  # conquer O(n)
  return merge(vL, vR, compare)

def merge(vL, vR, compare): # auxiliary O(n) space
  out = []
  i = j = 0
  # inserting
  while i < len(vL) and j < len(vR):
    if compare(vL[i], vR[j]):
      out.append(vL[i])
      i += 1
    else:
      out.append(vR[j])
      j += 1
  # extending
  while i < len(vL):
    out.append(vL[i])
    i += 1
  while j < len(vR):
    out.append(vR[j])
    j += 1

  return out

In [ ]:
def quicksort(v, l=0, r=None, compare=(lambda x, y: x < y)):  # inplace
  if r == None:
    r = len(v)-1
  if l < r:
    # divide (random)
    pidx = partition(v, l, r, compare)
    # conquer
    quicksort(v, l, pidx-1, compare)
    quicksort(v, pidx+1, r, compare)

def partition(v, l, r, compare):
  pivo = v[r] # random
  i = l
  for j in range(l,r):
    if compare(v[j], pivo):
      v[i], v[j] = v[j], v[i]
      i+=1
  v[i], v[r] = v[r], v[i]
  return i

In [ ]:
asc = lambda x, y: x < y
desc = lambda x, y: x > y

Aplicação:

In [ ]:
df_sort = df_full_clean.copy()

a = list(df_sort["last_available_confirmed_per_100k_inhabitants"])
b = list(df_sort["last_available_death_rate"])
c = list(df_sort["last_available_death_rate"])

a_ms = mergesort(a, compare=desc)
b_ms = mergesort(b, compare=desc)
c_ms = mergesort(c)

a_s = sorted(a, reverse=True)
b_s = sorted(b, reverse=True)
c_s = sorted(c)

quicksort(a, compare=desc)
quicksort(b, compare=desc)
quicksort(c)